#TAG SMART EMAILS

GENSIM AND LDA

https://radimrehurek.com/gensim/apiref.html

In [1]:
#Library
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import spacy
import pandas as pd

In [2]:
%pwd

'C:\\Users\\HomePC\\tagger-ds'

In [3]:
import requests
url = 'https://raw.githubusercontent.com/Moly-malibu/Tag-Smart-Emails/master/Enronemails.csv?token=AN2RW2FK3OPVHW7HTK5AKZS624Z7U'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

24004915

In [4]:
#EMAILS FROM ENRON COMPANY SAMPLING
df = pd.read_csv("email.csv").drop(['Unnamed: 0'], axis=1)
print(df.shape) 
df.head(7)

(10000, 2)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...


In [5]:
#TO GET EMAIL BODY
[s.strip() for s in df['message'][1].splitlines()]

['Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>',
 'Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: john.lavorato@enron.com',
 'Subject: Re:',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>',
 'X-cc:',
 'X-bcc:',
 "X-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen (Non-Privileged).pst',
 '',
 'Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.',
 '',
 'As far as the business meetings, I think it would be more productive to try and stimulate discussions across

In [6]:
email = ' '.join([s.strip() for s in df['message'][1].splitlines()][15:])

In [7]:
#CLEAN EMAIL BODY
def cleanemail(email):
    return ' '.join([s.strip() for s in email.splitlines()][15:])

df['email_body'] = df['message'].apply(cleanemail)

df.head()

,file,message,email_body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes th...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the sal..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [8]:
#CLEAN HTML
import re
import html

def cleanup_html(raw_html):
    clean_html = re.sub(r'<.*?>', '', raw_html)
    clean_html_http = re.sub(r'http\S+([\.]{3})?', '', clean_html)
    clean_txt = html.unescape(clean_html)
    return clean_txt
df['email_body'] = df['email_body'].apply(cleanup_html)

In [9]:
#CLEAN DATA
#remove whitespace
df['email_body'] = df['email_body'].str.strip().str.lower()

#Start with date
df['email_body'].str.match('\d?\d/\d?\d/\d{4}').all()

#Replace all non-overlapping matches
df['email_body'] = df['email_body'].str.replace('[^a-zA-Z\s]', '').str.replace('\s+', ' ')

In [10]:
import nltk
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
#Getting Stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# define additional stop words
import nltk
import spacy

nlp = spacy.load('en_core_web_lg')
my_stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 
                 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
                 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 
                 "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
                 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
                 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
                 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 
                 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
                 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
                 'into', 'through', 'during', 'before', 'after', 'above', 'below',
                 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
                 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
                 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 
                 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
                 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 
                 'can', 'will', 'just', 'don', "don't", 'should', "should've", 
                 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 
                 "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', 
                 "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', 
                 "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 
                 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 
                 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
                 "weren't", 'won', "won't", 'wouldn', "wouldn't"]

STOP_WORDS = nlp.Defaults.stop_words.union(my_stop_words)

stop = stopwords.words('english')
sno = SnowballStemmer('english')
print (stop)
print('***************************************')
print(sno.stem('tasty'))

STOP_WORDS = list(STOP_WORDS)
print('\n'.join([word for word in STOP_WORDS[:10]]))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [12]:
tokens = []
for doc in nlp.pipe(df['email_body'], batch_size=500):

    doc_tokens = []

    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.lemma_.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [13]:
df['tokens'].head()

0                                           [forecast]
1    [travel, business, meeting, take, fun, trip, e...
2                              [test, successful, way]
3    [randy, send, schedule, salary, level, schedul...
4                                [let, shoot, tuesday]
Name: tokens, dtype: object

In [14]:
id2word = Dictionary(df['tokens'])
print(len(id2word))

45973


In [15]:
id2word.filter_extremes(no_below=5, no_above=.98)
print(len(id2word))

14978


In [16]:
#https://radimrehurek.com/gensim/corpora/dictionary.html

corpus = [id2word.doc2bow(d) for d in df['tokens']]

In [17]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [18]:
model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, workers=12, passes=5)

In [19]:
import re

words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [20]:
topics = [' '.join(t[0:5]) for t in words]

In [21]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n")

------ Topic 0 ------
enron subject thank john contract
------ Topic 1 ------
john subject buy enron send
------ Topic 2 ------
image email message subject click
------ Topic 3 ------
email gas john request wine
------ Topic 4 ------
pm phillip subject new forward
------ Topic 5 ------
market iso john subject pm
------ Topic 6 ------
john subject phillip pm cc
------ Topic 7 ------
enron say company power market
------ Topic 8 ------
loan cost say company subject
------ Topic 9 ------
pm subject send email john


In [22]:
!pip install pyLDAvis
import pyLDAvis

In [23]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.141798 -0.003129       1        1  27.014420
6     -0.011389  0.076114       2        1  13.364538
2     -0.043526  0.056293       3        1  10.785114
0      0.035586  0.031039       4        1   9.362778
9     -0.113881  0.034605       5        1   8.965400
3     -0.071990 -0.122323       6        1   7.522446
5     -0.003576 -0.020676       7        1   6.782044
4      0.021141  0.019387       8        1   5.825484
1     -0.026096 -0.024857       9        1   5.817935
8      0.071931 -0.046453      10        1   4.559837, topic_info=         Term          Freq         Total Category  logprob  loglift
173        pm   8787.000000   8787.000000  Default  30.0000  30.0000
2674    image   3489.000000   3489.000000  Default  29.0000  29.0000
959     email   5898.000000   5898.000000  Default  28.0000  28.0000
208     enron  11436.000000  11436.000000  Default  27.0000  27.0000
193   subject   7895.000000   7895.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
692    credit    131.992462   1833.543213  Topic10  -6.0181   0.4566
125       day    137.110092   2796.380371  Topic10  -5.9800   0.0726
134    energy    140.000153   3672.959473  Topic10  -5.9592  -0.1792
141       gas    142.956833   4951.703613  Topic10  -5.9383  -0.4571
975      week    133.405716   3094.687744  Topic10  -6.0074  -0.0562

[936 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
90        1  0.032786       
90        2  0.342255       
90        3  0.134343       
90        4  0.037584       
90        5  0.173527       
...     ...       ...    ...
5226      6  0.023940  zdnet
5226      7  0.035911  zdnet
5226      8  0.005985  zdnet
5226      9  0.005985  zdnet
5226     10  0.023940  zdnet

[5001 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 3, 1, 10, 4, 6, 5, 2, 9])

In [24]:
#'corpus'

distro = [lda[d] for d in corpus]

num_topics = 20

def update(doc):
        d_dist = {k:0 for k in range(0,num_topics)}
        for topic in doc:
            d_dist[topic[0]] = topic[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [25]:
#Correlated
topics = pd.DataFrame.from_records(new_distro)
topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.033339,0.033338,0.533262,0.033338,0.033338,0.033339,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0.033338,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.254033,0.057943,0.000000,0.000000,0.000000,0.000000,0.000000,0.674670,0.000000,0,0,0,0,0
2,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.016672,0.545097,0.238171,0.016672,0,0,0,0,0
3,0.000000,0.000000,0.000000,0.000000,0.503627,0.000000,0.442187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
4,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.766621,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0.016670,0,0,0,0,0


In [26]:
df['email_body'] = topics.idxmax(axis=1)

#Topic  
df['email_body'].describe()

count    10000.000000
mean         8.099300
std          3.886174
min          0.000000
25%          5.000000
50%          8.000000
75%         12.000000
max         14.000000
Name: email_body, dtype: float64

In [27]:
df['email_body'].value_counts()

13    1737
9     1434
6     1064
5      893
4      857
8      782
12     717
1      462
10     440
14     392
7      290
11     257
3      242
2      218
0      215
Name: email_body, dtype: int64

Create the Emails Tags

Tfidf term frequency–inverse document frequency

In [28]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HomePC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from nltk.tokenize import word_tokenize

# split into words
tokens = word_tokenize(email)

# convert to lower case
tokens = [w.lower() for w in tokens]

# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))

# remove punctuation from each word
stripped = [re_punc.sub('', w) for w in tokens]

# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]

# filter out stop words
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:200])

['traveling', 'business', 'meeting', 'takes', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'business', 'plan', 'meetings', 'take', 'trip', 'without', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'get', 'honest', 'opinions', 'whether', 'trip', 'even', 'desired', 'necessary', 'far', 'business', 'meetings', 'think', 'would', 'productive', 'try', 'stimulate', 'discussions', 'across', 'different', 'groups', 'working', 'often', 'presenter', 'speaks', 'others', 'quiet', 'waiting', 'turn', 'meetings', 'might', 'better', 'held', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'austin', 'play', 'golf', 'rent', 'ski', 'boat', 'jet', 'ski', 'flying', 'somewhere', 'takes', 'much', 'time']


In [30]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=None)

count_train = count_vec.fit(words)
bag_of_words = count_vec.transform(words)

print(count_vec.get_feature_names())

['austin', 'better', 'boat', 'business', 'desired', 'different', 'discussion', 'discussions', 'especially', 'far', 'flying', 'formal', 'format', 'fun', 'golf', 'groups', 'held', 'holding', 'honest', 'jet', 'meeting', 'meetings', 'necessary', 'opinions', 'plan', 'play', 'prepare', 'presentation', 'presenter', 'productive', 'quiet', 'rent', 'round', 'ski', 'speaks', 'stimulate', 'suggest', 'suggestion', 'table', 'takes', 'think', 'time', 'traveling', 'trip', 'try', 'turn', 'waiting', 'working']


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_freque = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
txt_fitted = text_freque.fit(words)
txt_transformed = txt_fitted.transform(words)
print ("The text: ", words)

The text:  ['traveling', 'business', 'meeting', 'takes', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'business', 'plan', 'meetings', 'take', 'trip', 'without', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'get', 'honest', 'opinions', 'whether', 'trip', 'even', 'desired', 'necessary', 'far', 'business', 'meetings', 'think', 'would', 'productive', 'try', 'stimulate', 'discussions', 'across', 'different', 'groups', 'working', 'often', 'presenter', 'speaks', 'others', 'quiet', 'waiting', 'turn', 'meetings', 'might', 'better', 'held', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'austin', 'play', 'golf', 'rent', 'ski', 'boat', 'jet', 'ski', 'flying', 'somewhere', 'takes', 'much', 'time']


In [32]:
idfreque = text_freque.idf_
print(dict(zip(txt_fitted.get_feature_names(), idfreque)))

{'across': 5.31748811353631, 'austin': 5.31748811353631, 'better': 5.31748811353631, 'boat': 5.31748811353631, 'business': 3.9311937524164198, 'desired': 5.31748811353631, 'different': 5.31748811353631, 'discussion': 5.31748811353631, 'discussions': 5.31748811353631, 'especially': 5.31748811353631, 'even': 4.624340932976365, 'far': 5.31748811353631, 'flying': 5.31748811353631, 'formal': 5.31748811353631, 'format': 5.31748811353631, 'fun': 5.31748811353631, 'get': 5.31748811353631, 'go': 5.31748811353631, 'golf': 5.31748811353631, 'groups': 5.31748811353631, 'held': 5.31748811353631, 'holding': 5.31748811353631, 'honest': 5.31748811353631, 'jet': 5.31748811353631, 'meeting': 5.31748811353631, 'meetings': 3.9311937524164198, 'might': 5.31748811353631, 'much': 5.31748811353631, 'necessary': 5.31748811353631, 'often': 5.31748811353631, 'opinions': 5.31748811353631, 'others': 5.31748811353631, 'plan': 5.31748811353631, 'play': 5.31748811353631, 'prepare': 5.31748811353631, 'presentation': 5

In [33]:
text_freque.vocabulary_

{'traveling': 52,
 'business': 4,
 'meeting': 24,
 'takes': 49,
 'fun': 15,
 'trip': 53,
 'especially': 9,
 'prepare': 34,
 'presentation': 35,
 'would': 60,
 'suggest': 45,
 'holding': 21,
 'plan': 32,
 'meetings': 25,
 'take': 48,
 'without': 58,
 'formal': 13,
 'even': 10,
 'try': 54,
 'get': 16,
 'honest': 22,
 'opinions': 30,
 'whether': 57,
 'desired': 5,
 'necessary': 28,
 'far': 11,
 'think': 50,
 'productive': 37,
 'stimulate': 44,
 'discussions': 8,
 'across': 0,
 'different': 6,
 'groups': 19,
 'working': 59,
 'often': 29,
 'presenter': 36,
 'speaks': 43,
 'others': 31,
 'quiet': 38,
 'waiting': 56,
 'turn': 55,
 'might': 26,
 'better': 2,
 'held': 20,
 'round': 40,
 'table': 47,
 'discussion': 7,
 'format': 14,
 'suggestion': 46,
 'go': 17,
 'austin': 1,
 'play': 33,
 'golf': 18,
 'rent': 39,
 'ski': 41,
 'boat': 3,
 'jet': 23,
 'flying': 12,
 'somewhere': 42,
 'much': 27,
 'time': 51}

In [34]:
words_dist = dict(zip(txt_fitted.get_feature_names(), idfreque))
words_dist

{'across': 5.31748811353631,
 'austin': 5.31748811353631,
 'better': 5.31748811353631,
 'boat': 5.31748811353631,
 'business': 3.9311937524164198,
 'desired': 5.31748811353631,
 'different': 5.31748811353631,
 'discussion': 5.31748811353631,
 'discussions': 5.31748811353631,
 'especially': 5.31748811353631,
 'even': 4.624340932976365,
 'far': 5.31748811353631,
 'flying': 5.31748811353631,
 'formal': 5.31748811353631,
 'format': 5.31748811353631,
 'fun': 5.31748811353631,
 'get': 5.31748811353631,
 'go': 5.31748811353631,
 'golf': 5.31748811353631,
 'groups': 5.31748811353631,
 'held': 5.31748811353631,
 'holding': 5.31748811353631,
 'honest': 5.31748811353631,
 'jet': 5.31748811353631,
 'meeting': 5.31748811353631,
 'meetings': 3.9311937524164198,
 'might': 5.31748811353631,
 'much': 5.31748811353631,
 'necessary': 5.31748811353631,
 'often': 5.31748811353631,
 'opinions': 5.31748811353631,
 'others': 5.31748811353631,
 'plan': 5.31748811353631,
 'play': 5.31748811353631,
 'prepare': 5

In [35]:
import numpy as np

# get feature names
feature_names = np.array(text_freque.get_feature_names())
sorted_by_idf = np.argsort(text_freque.idf_)
print("Features with lowest id frequence:\n{}".format(feature_names[sorted_by_idf[:3]]))
print("\nFeatures with highest id frequence:\n{}".format(feature_names[sorted_by_idf[-3:]]))

Features with lowest id frequence:
['business' 'meetings' 'would']

Features with highest id frequence:
['often' 'get' 'opinions']


TFID VECTORIZER  - Transforms text to feature vectors


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
data = tfidf.fit_transform(words) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
data = pd.DataFrame(data.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
data.head()

,austin,better,boat,business,desired,different,discussion,discussions,especially,far,...,table,takes,think,time,traveling,trip,try,turn,waiting,working
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
#TOKENIZER AND LEMMA
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [40]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        tokenizer=tokenize)

# Create a vocabulary and get word counts per document
data = tfidf.fit_transform(words) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
data = pd.DataFrame(data.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
data.head()

,business,meeting,trip
0,0.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [41]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate Distance of TF-IDF Vectors
dist_matrix  = cosine_similarity(data)

# Turn it into a DataFrame
df = pd.DataFrame(dist_matrix)

In [42]:
words[5][:300]

'trip'

In [43]:
print(words[12][:130])

business


In [44]:
sum(data['business'])

4.0

Json and CSV

In [ ]:
def save_data_frame_as_csv(df=None, file=None):
    """
    Save a Pandas DataFrame to CSV format.
    """
    try:
        if not file.endswith('.csv'):
            file += '.csv'
        df.to_csv(file, index=False)
        print(f'DataFrame saved as: {file}')
    except Exception as e:
        print("DataFrame couldn't be saved: ", sys.exc_info()[0])
        raise

In [ ]:
save_data_frame_as_csv(df, './merged_data.csv')

In [ ]:
def save_data_frame_as_json(df=None, file=None, orient='records'):
  """
  Save a Pandas DataFrame to Json format.
  """
  try:
    if not file.endswith('.json'):
      file += '.json'
    df.to_json(file, orient=orient)
    print(f'Dataframe saved as: {file}')
  except Exception as e:
    print("Dataframe couldn't be saved: ", sys.exc_info()[0])
    raise

In [ ]:
save_data_frame_as_json(df, '.merged_data.json', orient='records')